In [1]:
import numpy as np
import pandas as pd
import gc
import os
import matplotlib.pyplot as plt
import polars as pl
from sklearn.metrics import mean_squared_error, mean_absolute_error
from joblib import Parallel, delayed
from more_itertools import chunked
from functools import reduce
from typing import List
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import pandas as pd
import joblib
import os


El objetivo de este Notebook es preparar los datos para modelar con redes neuronales

In [2]:
# Reducir memoria automáticamente
def optimizar_memoria(df):
    for col in df.select_dtypes(include=['int64', 'int32']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include=['float64', 'float32']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    return df

In [12]:
# Abrir el archivo parquet y cargarlo en un DataFrame data/l_vm_completa_train_pendientes.parquet
gc.collect()
df_full = pd.read_parquet('./data/l_vm_completa_train.parquet', engine='fastparquet')
df_pendientes = pd.read_parquet('./data/l_vm_completa_train_pendientes.parquet', engine='fastparquet')

In [13]:
# Concatenar los DataFrames df_full y df_pendientes por PRODUCT_ID,CUSTOMER_ID y PERIODO
df_full = pd.merge(
    df_full,
    df_pendientes,
    on=['PRODUCT_ID', 'CUSTOMER_ID', 'PERIODO'],
    how='inner'  # solo filas que existen en ambos DataFrames
)

In [15]:
print(df_full.columns.to_list())

['PERIODO', 'ANIO', 'MES', 'MES_SIN', 'MES_COS', 'TRIMESTRE', 'ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'CUSTOMER_ID', 'PRODUCT_ID', 'PLAN_PRECIOS_CUIDADOS', 'CUST_REQUEST_QTY', 'CUST_REQUEST_TN', 'TN', 'STOCK_FINAL', 'MEDIA_MOVIL_3M_CLI_PROD', 'MEDIA_MOVIL_6M_CLI_PROD', 'MEDIA_MOVIL_12M_CLI_PROD', 'DESVIO_MOVIL_3M_CLI_PROD', 'DESVIO_MOVIL_6M_CLI_PROD', 'DESVIO_MOVIL_12M_CLI_PROD', 'MEDIA_MOVIL_3M_PROD', 'MEDIA_MOVIL_6M_PROD', 'MEDIA_MOVIL_12M_PROD', 'DESVIO_MOVIL_3M_PROD', 'DESVIO_MOVIL_6M_PROD', 'DESVIO_MOVIL_12M_PROD', 'MEDIA_MOVIL_3M_CLI', 'MEDIA_MOVIL_6M_CLI', 'MEDIA_MOVIL_12M_CLI', 'DESVIO_MOVIL_3M_CLI', 'DESVIO_MOVIL_6M_CLI', 'DESVIO_MOVIL_12M_CLI', 'TN_LAG_01', 'TN_LAG_02', 'TN_LAG_03', 'TN_LAG_04', 'TN_LAG_05', 'TN_LAG_06', 'TN_LAG_07', 'TN_LAG_08', 'TN_LAG_09', 'TN_LAG_10', 'TN_LAG_11', 'TN_LAG_12', 'TN_LAG_13', 'TN_LAG_14', 'TN_LAG_15', 'CLASE', 'CLASE_DELTA', 'ORDINAL', 'TN_DELTA_01', 'TN_DELTA_02', 'TN_DELTA_03', 'TN_DELTA_04', 'TN_DELTA_05', 'TN_DELTA_06', 

In [16]:
df_full.columns.to_list()

['PERIODO',
 'ANIO',
 'MES',
 'MES_SIN',
 'MES_COS',
 'TRIMESTRE',
 'ID_CAT1',
 'ID_CAT2',
 'ID_CAT3',
 'ID_BRAND',
 'SKU_SIZE',
 'CUSTOMER_ID',
 'PRODUCT_ID',
 'PLAN_PRECIOS_CUIDADOS',
 'CUST_REQUEST_QTY',
 'CUST_REQUEST_TN',
 'TN',
 'STOCK_FINAL',
 'MEDIA_MOVIL_3M_CLI_PROD',
 'MEDIA_MOVIL_6M_CLI_PROD',
 'MEDIA_MOVIL_12M_CLI_PROD',
 'DESVIO_MOVIL_3M_CLI_PROD',
 'DESVIO_MOVIL_6M_CLI_PROD',
 'DESVIO_MOVIL_12M_CLI_PROD',
 'MEDIA_MOVIL_3M_PROD',
 'MEDIA_MOVIL_6M_PROD',
 'MEDIA_MOVIL_12M_PROD',
 'DESVIO_MOVIL_3M_PROD',
 'DESVIO_MOVIL_6M_PROD',
 'DESVIO_MOVIL_12M_PROD',
 'MEDIA_MOVIL_3M_CLI',
 'MEDIA_MOVIL_6M_CLI',
 'MEDIA_MOVIL_12M_CLI',
 'DESVIO_MOVIL_3M_CLI',
 'DESVIO_MOVIL_6M_CLI',
 'DESVIO_MOVIL_12M_CLI',
 'TN_LAG_01',
 'TN_LAG_02',
 'TN_LAG_03',
 'TN_LAG_04',
 'TN_LAG_05',
 'TN_LAG_06',
 'TN_LAG_07',
 'TN_LAG_08',
 'TN_LAG_09',
 'TN_LAG_10',
 'TN_LAG_11',
 'TN_LAG_12',
 'TN_LAG_13',
 'TN_LAG_14',
 'TN_LAG_15',
 'CLASE',
 'CLASE_DELTA',
 'ORDINAL',
 'TN_DELTA_01',
 'TN_DELTA_02',
 'TN_

In [17]:
# Buscar en df_full los product_id, customer_id que solo tienen ceros en TN
def buscar_productos_solo_ceros(df: pd.DataFrame) -> pd.DataFrame:
    grouped = df.groupby(['PRODUCT_ID', 'CUSTOMER_ID'])['TN'].sum().reset_index()
    productos_solo_ceros = grouped[grouped['TN'] == 0]
    return productos_solo_ceros

productos_solo_ceros = buscar_productos_solo_ceros(df_full)
print(f"🔍 Combinaciones PRODUCT_ID + CUSTOMER_ID con TN = 0 en todos sus registros: {len(productos_solo_ceros)}")

# Eliminar del df_full los product_id, customer_id que solo tienen ceros en TN
def eliminar_productos_solo_ceros(df: pd.DataFrame, productos_solo_ceros: pd.DataFrame) -> pd.DataFrame:
    productos_set = set(zip(productos_solo_ceros['PRODUCT_ID'], productos_solo_ceros['CUSTOMER_ID']))
    mask = df.set_index(['PRODUCT_ID', 'CUSTOMER_ID']).index.isin(productos_set)
    
    cantidad_eliminada = mask.sum()
    print(f"🗑️ Filas eliminadas de df_full: {cantidad_eliminada:,}")
    
    df_filtrado = df[~mask]
    return df_filtrado

df_full = eliminar_productos_solo_ceros(df_full, productos_solo_ceros)


🔍 Combinaciones PRODUCT_ID + CUSTOMER_ID con TN = 0 en todos sus registros: 327068
🗑️ Filas eliminadas de df_full: 6,594,430
🗑️ Filas eliminadas de df_full: 6,594,430


In [18]:
# Eliminar de df_full las filas donde la columna A_PREDECIR sea 'N'
df_full = df_full[df_full['A_PREDECIR'] != 'N']
df_full = df_full.drop(columns=['A_PREDECIR'])

In [19]:
# Por cada PERIODO mostrar la cantidad de filas
periodo_counts = df_full['PERIODO'].value_counts().sort_index()
print("📅 Cantidad de filas por PERIODO:")
print(periodo_counts)


📅 Cantidad de filas por PERIODO:
PERIODO
201701    153506
201702    169292
201703    173972
201704    175629
201705    177198
201706    179909
201707    184384
201708    187774
201709    190994
201710    195034
201711    201377
201712    201427
201801    201965
201802    202278
201803    203591
201804    207881
201805    212721
201806    213622
201807    215649
201808    216883
201809    222730
201810    226473
201811    229728
201812    230059
201901    230063
201902    230976
201903    235674
201904    244108
201905    247448
201906    251593
201907    256940
201908    260099
201909    262500
201910    262616
201911    262721
201912    262805
Name: count, dtype: int64


In [20]:
# Agregar una columna que indique la diferencia en ORDINAL entre el ORDINAL actual y el ORDINAL anterior donde TN sea mayor a 0
# para ese CUSTOMER_ID y PRODUCT_ID

from joblib import Parallel, delayed
import numpy as np
import pandas as pd

def calcular_mejoras_por_grupo(grupo):
    grupo = grupo.sort_values('ORDINAL').copy()
    ult_ordinal = None
    valores = []

    for _, row in grupo.iterrows():
        if ult_ordinal is None:
            valores.append(36)
        else:
            valores.append(int(row['ORDINAL'] - ult_ordinal))

        if row['TN'] > 0:
            ult_ordinal = row['ORDINAL']

    grupo['MESES_SIN_COMPRAR_PRODUCT_CUSTOMER_ID'] = np.array(valores, dtype=np.int16)
    return grupo

def agregar_diferencia_ordinal_parallel(df: pd.DataFrame, n_jobs: int = -1) -> pd.DataFrame:
    df = df.copy()
    df['MESES_SIN_COMPRAR_PRODUCT_CUSTOMER_ID'] = 36  # valor inicial
    df['MESES_SIN_COMPRAR_PRODUCT_CUSTOMER_ID'] = df['MESES_SIN_COMPRAR_PRODUCT_CUSTOMER_ID'].astype('int16')

    # Agrupar por cliente y producto
    grupos = list(df.groupby(['CUSTOMER_ID', 'PRODUCT_ID']))

    # Procesar en paralelo
    resultados = Parallel(n_jobs=n_jobs, backend='loky', batch_size=128)(
        delayed(calcular_mejoras_por_grupo)(grupo) for _, grupo in grupos
    )

    # Concatenar todos los resultados
    df_resultado = pd.concat(resultados, axis=0).sort_index()
    return df_resultado



df_full = agregar_diferencia_ordinal_parallel(df_full, n_jobs=28)

In [21]:
from joblib import Parallel, delayed
import numpy as np
import pandas as pd

def calcular_mejoras_por_producto(grupo):
    grupo = grupo.sort_values('ORDINAL').copy()
    ult_ordinal = None
    valores = []

    for _, row in grupo.iterrows():
        if ult_ordinal is None:
            valores.append(36)
        else:
            valores.append(int(row['ORDINAL'] - ult_ordinal))

        if row['TN'] > 0:
            ult_ordinal = row['ORDINAL']

    grupo['MESES_SIN_COMPRAR_PRODUCT_ID'] = np.array(valores, dtype=np.int16)
    return grupo

def agregar_diferencia_ordinal_por_producto(df: pd.DataFrame, n_jobs: int = -1) -> pd.DataFrame:
    df = df.copy()
    df['MESES_SIN_COMPRAR_PRODUCT_ID'] = 36
    df['MESES_SIN_COMPRAR_PRODUCT_ID'] = df['MESES_SIN_COMPRAR_PRODUCT_ID'].astype('int16')

    # Agrupar solo por PRODUCT_ID
    grupos = list(df.groupby('PRODUCT_ID'))

    resultados = Parallel(n_jobs=n_jobs, backend='loky', batch_size=128)(
        delayed(calcular_mejoras_por_producto)(grupo) for _, grupo in grupos
    )

    df_resultado = pd.concat(resultados, axis=0).sort_index()
    return df_resultado

df_full = agregar_diferencia_ordinal_por_producto(df_full, n_jobs=28)


In [22]:
from joblib import Parallel, delayed
import numpy as np
import pandas as pd

def calcular_mejoras_por_cliente(grupo):
    grupo = grupo.sort_values('ORDINAL').copy()
    ult_ordinal = None
    valores = []

    for _, row in grupo.iterrows():
        if ult_ordinal is None:
            valores.append(36)
        else:
            valores.append(int(row['ORDINAL'] - ult_ordinal))

        if row['TN'] > 0:
            ult_ordinal = row['ORDINAL']

    grupo['MESES_SIN_COMPRAR_CUSTOMER_ID'] = np.array(valores, dtype=np.int16)
    return grupo

def agregar_diferencia_ordinal_por_cliente(df: pd.DataFrame, n_jobs: int = -1) -> pd.DataFrame:
    df = df.copy()
    df['MESES_SIN_COMPRAR_CUSTOMER_ID'] = 36
    df['MESES_SIN_COMPRAR_CUSTOMER_ID'] = df['MESES_SIN_COMPRAR_CUSTOMER_ID'].astype('int16')

    grupos = list(df.groupby('CUSTOMER_ID'))

    resultados = Parallel(n_jobs=n_jobs, backend='loky', batch_size=128)(
        delayed(calcular_mejoras_por_cliente)(grupo) for _, grupo in grupos
    )

    df_resultado = pd.concat(resultados, axis=0).sort_index()
    return df_resultado

df_full = agregar_diferencia_ordinal_por_cliente(df_full, n_jobs=28)

In [23]:
# Agregar a df_full una variable categorica MES_PROBLEMATICO que sea 1 si PERIODO es 201906 o 201908 o 201910, y 0 en caso contrario
# Calcular los días del mes usando las columnas ANIO y MES

# Agregar a df_full una variable categorica MES_PROBLEMATICO que sea 1 si ANIO==2019 y MES en [6, 8, 10], y 0 en caso contrario
df_full['MES_PROBLEMATICO'] = np.where(
       (df_full['ANIO'] == 2019) & (df_full['MES'].isin([6, 8, 10])),
       1., 0.0
)
df_full['MES_PROBLEMATICO'] = df_full['MES_PROBLEMATICO'].astype(np.float32)

In [24]:
# Guardar el DataFrame en parquet
df_full.to_parquet('./data/interm_NN_TORCH.parquet', index=False, engine='fastparquet')

In [25]:
print(df_full.columns.to_list())

['PERIODO', 'ANIO', 'MES', 'MES_SIN', 'MES_COS', 'TRIMESTRE', 'ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'CUSTOMER_ID', 'PRODUCT_ID', 'PLAN_PRECIOS_CUIDADOS', 'CUST_REQUEST_QTY', 'CUST_REQUEST_TN', 'TN', 'STOCK_FINAL', 'MEDIA_MOVIL_3M_CLI_PROD', 'MEDIA_MOVIL_6M_CLI_PROD', 'MEDIA_MOVIL_12M_CLI_PROD', 'DESVIO_MOVIL_3M_CLI_PROD', 'DESVIO_MOVIL_6M_CLI_PROD', 'DESVIO_MOVIL_12M_CLI_PROD', 'MEDIA_MOVIL_3M_PROD', 'MEDIA_MOVIL_6M_PROD', 'MEDIA_MOVIL_12M_PROD', 'DESVIO_MOVIL_3M_PROD', 'DESVIO_MOVIL_6M_PROD', 'DESVIO_MOVIL_12M_PROD', 'MEDIA_MOVIL_3M_CLI', 'MEDIA_MOVIL_6M_CLI', 'MEDIA_MOVIL_12M_CLI', 'DESVIO_MOVIL_3M_CLI', 'DESVIO_MOVIL_6M_CLI', 'DESVIO_MOVIL_12M_CLI', 'TN_LAG_01', 'TN_LAG_02', 'TN_LAG_03', 'TN_LAG_04', 'TN_LAG_05', 'TN_LAG_06', 'TN_LAG_07', 'TN_LAG_08', 'TN_LAG_09', 'TN_LAG_10', 'TN_LAG_11', 'TN_LAG_12', 'TN_LAG_13', 'TN_LAG_14', 'TN_LAG_15', 'CLASE', 'CLASE_DELTA', 'ORDINAL', 'TN_DELTA_01', 'TN_DELTA_02', 'TN_DELTA_03', 'TN_DELTA_04', 'TN_DELTA_05', 'TN_DELTA_06', 

In [26]:

# Conservar las siguientes columnas
# #columns_to_keep = ['MES_SIN', 'MES_COS', 'ID_CAT1',
#        'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'CUSTOMER_ID',
#        'PRODUCT_ID', 'CUST_REQUEST_QTY',
#        'CUST_REQUEST_TN', 'TN', 'CLASE_DELTA',
#        'ANTIG_CLIENTE','ANTIG_PRODUCTO', 'CANT_PROD_CLI_PER',
#        'MESES_SIN_COMPRAR_PRODUCT_CUSTOMER_ID','MESES_SIN_COMPRAR_PRODUCT_ID','MESES_SIN_COMPRAR_CUSTOMER_ID',
#        'MES_PROBLEMATICO','PERIODO','ORDINAL']
# Filtrar el DataFrame para conservar solo las columnas deseadas 
columns_to_keep = ['PERIODO', 'ANIO', 'MES', 'MES_SIN', 'MES_COS', 'TRIMESTRE', 'ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 
                   'SKU_SIZE', 'CUSTOMER_ID', 'PRODUCT_ID', 'CUST_REQUEST_QTY', 'CUST_REQUEST_TN', 'TN', 
                   'MEDIA_MOVIL_3M_CLI_PROD', 'MEDIA_MOVIL_6M_CLI_PROD', 'MEDIA_MOVIL_12M_CLI_PROD', 
                   'DESVIO_MOVIL_3M_CLI_PROD', 'DESVIO_MOVIL_6M_CLI_PROD', 'DESVIO_MOVIL_12M_CLI_PROD', 'MEDIA_MOVIL_3M_PROD', 
                   'MEDIA_MOVIL_6M_PROD', 'MEDIA_MOVIL_12M_PROD', 'DESVIO_MOVIL_3M_PROD', 'DESVIO_MOVIL_6M_PROD', 
                   'DESVIO_MOVIL_12M_PROD', 'MEDIA_MOVIL_3M_CLI', 'MEDIA_MOVIL_6M_CLI', 'MEDIA_MOVIL_12M_CLI', 
                   'DESVIO_MOVIL_3M_CLI', 'DESVIO_MOVIL_6M_CLI', 'DESVIO_MOVIL_12M_CLI', 'TN_LAG_01', 'TN_LAG_02', 'TN_LAG_03', 
                   'TN_LAG_04', 'TN_LAG_05', 'TN_LAG_06', 'TN_LAG_07', 'TN_LAG_08', 'TN_LAG_09', 'TN_LAG_10', 'TN_LAG_11', 
                   'TN_LAG_12', 'TN_LAG_13', 'TN_LAG_14', 'TN_LAG_15','CLASE_DELTA', 'ORDINAL', 
                   'TN_DELTA_01', 'TN_DELTA_02', 'TN_DELTA_03', 'TN_DELTA_04', 'TN_DELTA_05', 'TN_DELTA_06', 'TN_DELTA_07', 
                   'TN_DELTA_08', 'TN_DELTA_09', 'TN_DELTA_10', 'TN_DELTA_11', 'TN_DELTA_12', 'TN_DELTA_13', 'TN_DELTA_14', 
                   'TN_DELTA_15', 'ANTIG_CLIENTE', 'ANTIG_PRODUCTO', 'CANT_PROD_CLI_PER', 'MEDIA_PROD_PER', 'MEDIA_PROD', 
                   'MEDIA_PER', 'PENDIENTE_TENDENCIA_3', 'TN_EWMA_03', 'TN_MEDIAN_03', 'TN_MIN_03', 'TN_MAX_03', 
                   'PENDIENTE_TENDENCIA_6', 'TN_EWMA_06', 'TN_MEDIAN_06', 'TN_MIN_06', 'TN_MAX_06', 'PENDIENTE_TENDENCIA_9',
                   'TN_EWMA_09', 'TN_MEDIAN_09', 'TN_MIN_09', 'TN_MAX_09', 'PENDIENTE_TENDENCIA_12', 'TN_EWMA_12', 
                   'TN_MEDIAN_12', 'TN_MIN_12', 'TN_MAX_12', 'MESES_SIN_COMPRAR_PRODUCT_CUSTOMER_ID', 
                   'MESES_SIN_COMPRAR_PRODUCT_ID', 'MESES_SIN_COMPRAR_CUSTOMER_ID', 'MES_PROBLEMATICO']
df_full = df_full[columns_to_keep]

In [27]:
def log_transform_signed(x):
    return np.sign(x) * np.log1p(np.abs(x))

df_full['CLASE_DELTA_LOG1P'] = log_transform_signed(df_full['CLASE_DELTA'])


In [28]:
from sklearn.preprocessing import StandardScaler

scaler_y = StandardScaler()
df_full['CLASE_DELTA_LOG1P_Z'] = scaler_y.fit_transform(df_full[['CLASE_DELTA_LOG1P']])

media_y = scaler_y.mean_[0]
std_y = scaler_y.scale_[0]

print(f"Media de CLASE_DELTA_LOG1P_Z: {media_y}")
print(f"Desviación estándar de CLASE_DELTA_LOG1P_Z: {std_y}")



Media de CLASE_DELTA_LOG1P_Z: -0.0002512964473318228
Desviación estándar de CLASE_DELTA_LOG1P_Z: 0.2535427832171512


In [29]:
def revert_clase_delta_log1p_z(pred_z, media = media_y, std = std_y):
    pred_log1p = pred_z * std + media
    return np.sign(pred_log1p) * (np.expm1(np.abs(pred_log1p)))


In [30]:
# Eliminar las columnas CLASE_DELTA y CLASE_DELTA_LOG1P para evitar confusiones
df_full.drop(columns=['CLASE_DELTA', 'CLASE_DELTA_LOG1P'], inplace=True, errors='ignore')

In [31]:


cat_cols = ['ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE','ANIO', 'MES','TRIMESTRE','MES_PROBLEMATICO']

for col in cat_cols:
    if df_full[col].isnull().any():
        df_full[col] = df_full[col].fillna("missing")


encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    df_full.loc[:, col] = le.fit_transform(df_full[col]).astype(int)
    encoders[col] = le  # para guardar los mapeos por si necesitás revertirlos


In [32]:
# Guardo los encoders en archivos .pkl
import joblib
import os

os.makedirs('encoders', exist_ok=True)

for col, le in encoders.items():
    joblib.dump(le, f'encoders/{col}_encoder.pkl')

In [33]:
# Guardar el DataFrame en parquet
df_full.to_parquet('./data/interm_NN_TORCH.parquet', index=False, engine='fastparquet')

In [2]:
df_full = pd.read_parquet('./data/interm_NN_TORCH.parquet', index=False, engine='fastparquet')

In [ ]:
""" import numpy as np
import pandas as pd
from scipy.stats import linregress
from concurrent.futures import ProcessPoolExecutor, as_completed
import gc
from tqdm import tqdm


def process_group(group_key, df_group, columnas, lags, rolling_windows, trend_windows, time_col):
    df_group = df_group.sort_values(by=time_col)
    df_group_result = df_group.copy()
    nuevas_columnas = []

    for col in columnas:
        col_data = df_group[col]
        min_val = col_data[col_data > 0].min() if (col_data > 0).any() else 1

        # Lags y Deltas para todas las columnas
        for lag in lags:
            lag_col = f'{col}_LAG_{lag:02d}'
            delta_col = f'{col}_DELTA_{lag:02d}'
            delta_pct_col = f'{col}_DELTA_PCT_{lag:02d}'

            df_group_result[lag_col] = col_data.shift(lag)
            df_group_result[delta_col] = col_data - df_group_result[lag_col]
            reemplazo = np.where(df_group_result[lag_col] == 0, min_val, df_group_result[lag_col])
            df_group_result[delta_pct_col] = col_data / reemplazo
            df_group_result[delta_pct_col] = df_group_result[delta_pct_col].replace([np.inf, -np.inf], 0)

            nuevas_columnas += [lag_col, delta_col, delta_pct_col]

        # Rolling y tendencias solo para 'TN'
        if col == 'TN':
            for window in rolling_windows:
                roll_mean_col = f'{col}_ROLL_MEAN_{window}'
                roll_std_col = f'{col}_ROLL_STD_{window}'
                roll_z_col = f'{col}_ROLL_Z_{window}'

                roll_mean = col_data.rolling(window, min_periods=1).mean()
                roll_std = col_data.rolling(window, min_periods=1).std(ddof=0).fillna(0)
                roll_z = (col_data - roll_mean) / roll_std.replace(0, 1)

                df_group_result[roll_mean_col] = roll_mean
                df_group_result[roll_std_col] = roll_std
                df_group_result[roll_z_col] = roll_z

                nuevas_columnas += [roll_mean_col, roll_std_col, roll_z_col]

            for window in trend_windows:
                slope_col = f'{col}_SLOPE_{window}'
                def slope_func(x):
                    if len(x) < 2:
                        return 0.0
                    x_ = np.arange(len(x))
                    return linregress(x_, x).slope
                slope = col_data.rolling(window, min_periods=2).apply(slope_func, raw=True)
                df_group_result[slope_col] = slope
                nuevas_columnas.append(slope_col)

    df_group_result[nuevas_columnas] = df_group_result[nuevas_columnas].fillna(0)
    return df_group_result

def generar_lags_deltas_rolling_parallel_optim(
    df: pd.DataFrame,
    columnas: list,
    lags: list = list(range(1, 4)),
    rolling_windows: list = [3, 6],
    trend_windows: list = [6],
    group_cols: list = ['PRODUCT_ID', 'CUSTOMER_ID'],
    time_col: str = 'ORDINAL',
    n_workers: int = 20,
    batch_size: int = 50
):
    print("🔄 Iniciando procesamiento optimizado...")
    df = df.sort_values(by=group_cols + [time_col])
    grouped = df.groupby(group_cols)
    total_groups = grouped.ngroups
    print(f"📊 Procesando {total_groups:,} grupos en lotes de {batch_size}")
    all_results = []
    processed_groups = 0
    group_iter = iter(grouped)
    while processed_groups < total_groups:
        batch = []
        for _ in range(min(batch_size, total_groups - processed_groups)):
            try:
                batch.append(next(group_iter))
            except StopIteration:
                break
        if not batch:
            break
        with ProcessPoolExecutor(max_workers=n_workers) as executor:
            futures = []
            for name, group in batch:
                group_subset = group[group_cols + [time_col] + columnas].copy()
                future = executor.submit(
                    process_group,
                    name,
                    group_subset,
                    columnas,
                    lags,
                    rolling_windows,
                    trend_windows,
                    time_col
                )
                futures.append(future)
            batch_results = []
            # Mostrar progreso dentro del batch
            for future in tqdm(as_completed(futures), total=len(futures), desc=f"📦 Procesando batch ({processed_groups}/{total_groups})"):
                try:
                    result = future.result()
                    batch_results.append(result)
                except Exception as e:
                    print(f"❌ Error procesando grupo: {e}")
        if batch_results:
            batch_df = pd.concat(batch_results, ignore_index=True)
            all_results.append(batch_df)
            del batch_results, batch_df
            gc.collect()
        processed_groups += len(batch)
        print(f"✅ Procesados {processed_groups:,}/{total_groups:,} grupos ({processed_groups/total_groups*100:.1f}%)")
    print("🔄 Concatenando resultados finales...")
    df_final = pd.concat(all_results, ignore_index=True)
    print("🔄 Optimizando tipos de datos...")
    for col in df_final.columns:
        if col not in group_cols + [time_col] and pd.api.types.is_numeric_dtype(df_final[col]):
            if df_final[col].dtype == 'float64':
                df_final[col] = df_final[col].astype(np.float32)
            elif df_final[col].dtype in ['int64', 'int32']:
                if df_final[col].min() >= -32768 and df_final[col].max() <= 32767:
                    df_final[col] = df_final[col].astype(np.int16)
    del all_results
    gc.collect()
    print(f"✅ Procesamiento completado. Shape final: {df_final.shape}")
    return df_final.sort_index()
 """

In [ ]:
""" df_full = generar_lags_deltas_rolling_parallel_optim(
    df=df_full,
    columnas=['TN', 'CUST_REQUEST_QTY', 'CUST_REQUEST_TN', 'CANT_PROD_CLI_PER'],
    lags=list(range(1, 12)),
    rolling_windows=[3, 6],
    trend_windows=[6],
    n_workers=28,
    batch_size=50000  # Bajalo si aún tenés problemas de RAM
) """

In [34]:
# Guardar el DataFrame en parquet
df_full.to_parquet('./data/interm_NN_TORCH.parquet', index=False, engine='fastparquet')

In [35]:
# Excluir la variable de clase
cols_to_check = df_full.columns.difference(['CLASE_DELTA_LOG1P_Z'])

# Calcular cantidad de NaNs por columna
nan_columns = df_full[cols_to_check].isna().sum()

# Filtrar solo las columnas que tienen al menos un NaN
nan_columns = nan_columns[nan_columns > 0].sort_values(ascending=False)

# Mostrar
print(nan_columns)

PENDIENTE_TENDENCIA_12    2777480
TN_EWMA_12                2777480
TN_MEDIAN_12              2777480
TN_MAX_12                 2777480
TN_MIN_12                 2777480
TN_EWMA_09                2066722
PENDIENTE_TENDENCIA_9     2066722
TN_MEDIAN_09              2066722
TN_MIN_09                 2066722
TN_MAX_09                 2066722
TN_EWMA_06                1310741
TN_MAX_06                 1310741
TN_MEDIAN_06              1310741
PENDIENTE_TENDENCIA_6     1310741
TN_MIN_06                 1310741
TN_EWMA_03                 525526
TN_MAX_03                  525526
PENDIENTE_TENDENCIA_3      525526
TN_MEDIAN_03               525526
TN_MIN_03                  525526
dtype: int64


In [37]:
# Para las columnas con NaN, si son PENDIENTE_TENDENCIA_nn reemplazar por 0,
# sin son TN_EWMA reemplazar por la media de la columna
for col in nan_columns.index:
    if 'PENDIENTE_TENDENCIA' in col:
        df_full[col].fillna(0, inplace=True)
    elif 'TN_EWMA' in col:
        media_col = df_full[col].mean()
        df_full[col].fillna(media_col, inplace=True)
    elif 'TN_MIN' in col or 'TN_MAX' in col or 'TN_MEDIAN' in col:
        media_col = df_full[col].mean()
        df_full[col].fillna(media_col, inplace=True)
    

/tmp/ipykernel_3921255/2542019331.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_full[col].fillna(0, inplace=True)
/tmp/ipykernel_3921255/2542019331.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 

In [39]:
# Excluir la variable de clase
cols_to_check = df_full.columns.difference(['CLASE_DELTA_LOG1P_Z'])

# Calcular cantidad de NaNs por columna
nan_columns = df_full[cols_to_check].isna().sum()

# Filtrar solo las columnas que tienen al menos un NaN
nan_columns = nan_columns[nan_columns > 0].sort_values(ascending=False)

# Mostrar
print(nan_columns)

Series([], dtype: int64)


In [40]:
print(f"📊 DataFrame final con {df_full.shape[0]:,} filas y {df_full.shape[1]} columnas:")

📊 DataFrame final con 7,781,619 filas y 96 columnas:


In [41]:
# Copia de seguridad
df = df_full.copy()


# === Binning (en deciles) ===
df['CUSTOMER_RANK_BIN'] = pd.qcut(df['CUSTOMER_ID'], q=10, labels=False)
df['CUSTOMER_RANK_BIN'] = df['CUSTOMER_RANK_BIN'].astype('category')

# Validación opcional
assert df['CUSTOMER_RANK_BIN'].isna().sum() == 0, "NaNs en qcut"

# Reemplazar en df_full
df_full = df



In [42]:
# Copia de seguridad del DataFrame
df = df_full.copy()

# === Binning (en deciles) ===
df['PRODUCT_RANK_BIN'] = pd.qcut(df['PRODUCT_ID'], q=10, labels=False)
df['PRODUCT_RANK_BIN'] = df['PRODUCT_RANK_BIN'].astype('category')

# Reemplazar en el DataFrame principal
df_full = df


In [43]:
# Copia del DataFrame original
df = df_full.copy()

# Columnas categóricas
cat_cols = cat_cols + ['CUSTOMER_RANK_BIN', 'PRODUCT_RANK_BIN']


In [44]:
print(cat_cols)

['ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'ANIO', 'MES', 'TRIMESTRE', 'MES_PROBLEMATICO', 'CUSTOMER_RANK_BIN', 'PRODUCT_RANK_BIN']


In [ ]:

# Columnas a excluir
excluir = ['PERIODO','CLASE_DELTA_LOG1P_Z', 'MES_SIN', 'MES_COS'] + cat_cols
print(excluir)

['PERIODO', 'CLASE_DELTA_LOG1P_Z', 'MES_SIN', 'MES_COS', 'ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'ANIO', 'MES', 'TRIMESTRE', 'MES_PROBLEMATICO', 'CUSTOMER_RANK_BIN', 'PRODUCT_RANK_BIN', 'CUSTOMER_RANK_BIN', 'PRODUCT_RANK_BIN']


In [46]:

# Columnas numéricas a escalar
cols_a_escalar = [col for col in df.columns if col not in excluir and pd.api.types.is_numeric_dtype(df[col])]

# Entrenamiento del scaler SOLO con datos de entrenamiento
df_entrenamiento = df[df['PERIODO'] <= 201910].copy()
scaler = StandardScaler()
scaler.fit(df_entrenamiento[cols_a_escalar])

# Aplicación del scaler a TODO el dataset
valores_escalados = scaler.transform(df[cols_a_escalar])
df_scaled = pd.DataFrame(valores_escalados, columns=[col + '_Z' for col in cols_a_escalar], index=df.index)

# Reemplazo de columnas originales salvo CUSTOMER_ID y PRODUCT_ID
cols_a_escalar = [col for col in cols_a_escalar if col not in ['CUSTOMER_ID', 'PRODUCT_ID']]
df.drop(columns=cols_a_escalar, inplace=True)
df = pd.concat([df, df_scaled], axis=1)

# Guardar scaler
joblib.dump(scaler, './encoders/scaler_features_numericas.pkl')

# Actualizar df_full
df_full = df


In [47]:
print(df_full.columns.tolist())

['PERIODO', 'ANIO', 'MES', 'MES_SIN', 'MES_COS', 'TRIMESTRE', 'ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'CUSTOMER_ID', 'PRODUCT_ID', 'MES_PROBLEMATICO', 'CLASE_DELTA_LOG1P_Z', 'CUSTOMER_RANK_BIN', 'PRODUCT_RANK_BIN', 'CUSTOMER_ID_Z', 'PRODUCT_ID_Z', 'CUST_REQUEST_QTY_Z', 'CUST_REQUEST_TN_Z', 'TN_Z', 'MEDIA_MOVIL_3M_CLI_PROD_Z', 'MEDIA_MOVIL_6M_CLI_PROD_Z', 'MEDIA_MOVIL_12M_CLI_PROD_Z', 'DESVIO_MOVIL_3M_CLI_PROD_Z', 'DESVIO_MOVIL_6M_CLI_PROD_Z', 'DESVIO_MOVIL_12M_CLI_PROD_Z', 'MEDIA_MOVIL_3M_PROD_Z', 'MEDIA_MOVIL_6M_PROD_Z', 'MEDIA_MOVIL_12M_PROD_Z', 'DESVIO_MOVIL_3M_PROD_Z', 'DESVIO_MOVIL_6M_PROD_Z', 'DESVIO_MOVIL_12M_PROD_Z', 'MEDIA_MOVIL_3M_CLI_Z', 'MEDIA_MOVIL_6M_CLI_Z', 'MEDIA_MOVIL_12M_CLI_Z', 'DESVIO_MOVIL_3M_CLI_Z', 'DESVIO_MOVIL_6M_CLI_Z', 'DESVIO_MOVIL_12M_CLI_Z', 'TN_LAG_01_Z', 'TN_LAG_02_Z', 'TN_LAG_03_Z', 'TN_LAG_04_Z', 'TN_LAG_05_Z', 'TN_LAG_06_Z', 'TN_LAG_07_Z', 'TN_LAG_08_Z', 'TN_LAG_09_Z', 'TN_LAG_10_Z', 'TN_LAG_11_Z', 'TN_LAG_12_Z', 'TN_LAG_13_Z', 'TN_LA

In [48]:
# Excluir la variable de clase
cols_to_check = df_full.columns.difference(['CLASE_DELTA_LOG1P_Z'])

# Calcular cantidad de NaNs por columna
nan_columns = df_full[cols_to_check].isna().sum()

# Filtrar solo las columnas que tienen al menos un NaN
nan_columns = nan_columns[nan_columns > 0].sort_values(ascending=False)

# Mostrar
print(nan_columns)

Series([], dtype: int64)


In [49]:
print(df_full.shape)
# Guardar el DataFrame resultante en un archivo parquet
df_full.to_parquet('./data/train_val_NN_TORCH.parquet', engine='fastparquet', index=False)

(7781619, 100)


In [50]:
print(cat_cols)

['ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'ANIO', 'MES', 'TRIMESTRE', 'MES_PROBLEMATICO', 'CUSTOMER_RANK_BIN', 'PRODUCT_RANK_BIN']
